In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

ModuleNotFoundError: No module named 'sklearn'

In [2]:
data = pd.read_csv('caesarian.csv')
non_categorical=['Age']
decision_column_name='Caesarian'
count_of_yes_no=[]
tables={}

In [4]:
train, test = train_test_split(data, test_size=.20, random_state=10)
print(train)

NameError: name 'train_test_split' is not defined